In [ ]:
!pip install -q -U torch
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import bitsandbytes as bnb
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training,PeftConfig,PeftModel
import torch
from transformers import AutoModelForCausalLM,LlamaForCausalLM,LlamaTokenizer, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
   DataCollatorForLanguageModeling, Trainer, TrainingArguments, TextStreamer
from datasets import load_dataset,Dataset

In [ ]:
# Loading Model and Tokenizer with a GPU limit of at most 8 GB
def load_model(model_name, bnb_config):
   n_gpus = torch.cuda.device_count()
   max_memory = f'{8000}MB'


   model = LlamaForCausalLM.from_pretrained(
       model_name,
       quantization_config=bnb_config,
       device_map="auto",  # Efficiently dispatch the model on available resources
       max_memory={i: max_memory for i in range(n_gpus)},
       cache_dir=cache_dir
   )
   tokenizer = LlamaTokenizer.from_pretrained(model_name, cache_dir=cache_dir)


   # Needed for LLaMA tokenizer
   tokenizer.pad_token = tokenizer.eos_token


   return model, tokenizer

In [ ]:

# Create a BitsAndBytesConfig for quantization
def create_bnb_config():
   # Configure BitsAndBytes quantization with specific settings
   bnb_config = BitsAndBytesConfig(
       load_in_4bit=True,                    # Load weights in 4-bit format
       bnb_4bit_use_double_quant=True,       # Use double quantization for 4-bit
       bnb_4bit_quant_type="nf4",           # 4-bit quantization type
       bnb_4bit_compute_dtype=torch.bfloat16, # Compute data type for 4-bit
   )


   return bnb_config


# Create a Parameter-Efficient Fine-Tuning config for your model
def create_peft_config(modules):
   """
   Create Parameter-Efficient Fine-Tuning config for your model
   :param modules: Names of the modules to apply Lora to
   """
   # Configure Lora (Parameter-Efficient Fine-Tuning) with specific settings
   config = LoraConfig(
       r=16,                # Dimension of the updated matrices
       lora_alpha=64,       # Parameter for scaling
       target_modules=modules, # Names of the modules to apply Lora to
       lora_dropout=0.05,    # Dropout probability
       bias="none",         # Bias type
       task_type="CAUSAL_LM", # Task type (Causal Language Modeling in this case)
   )


   return config

In [ ]:
def create_prompt_formats(sample):
   """
   Format various fields of the sample ('instruction', 'context', 'response')
   Then concatenate them using two newline characters
   :param sample: Sample dictionary
   """


   system_prompt = '''तुम एक सहायक हो जो सटीक और संक्षेपित उत्तर प्रदान करता है। कृपया प्रदान किए गए पाठ में सूचना ढूंढ़ें और सवाल का संक्षेप में उत्तर दें। अगर आपको उत्तर नहीं पता है, तो एक से ज्यादा वाक्य में बस बताएं कि आप नहीं जानते।'''


   B_INST, E_INST = "[INST]", "[/INST]"
   B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n"


   user_prompt = sample['question']
   context = sample['context']
   response = sample['answer']

   prompt = f"{B_INST} {B_SYS} {system_prompt.strip()} {E_SYS} \n संदर्भ: {context.strip()} \n प्रश्न: {user_prompt.strip()} {E_INST} \n\n  उत्तर: {response}"


   return prompt


def generate_and_tokenize_prompt(data_point):
   full_prompt = create_prompt_formats(data_point)
   tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True,max_length=1024)
   return tokenized_full_prompt

In [ ]:
def find_all_linear_names(model):
   cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
   lora_module_names = set()
   for name, module in model.named_modules():
       if isinstance(module, cls):
           names = name.split('.')
           lora_module_names.add(names[0] if len(names) == 1 else names[-1])


   if 'lm_head' in lora_module_names:  # needed for 16-bit
       lora_module_names.remove('lm_head')
   return list(lora_module_names)


def print_trainable_parameters(model, use_4bit=False):
   """
   Prints the number of trainable parameters in the model.
   """
   trainable_params = 0
   all_param = 0
   for _, param in model.named_parameters():
       num_params = param.numel()
       # if using DS Zero 3 and the weights are initialized empty
       if num_params == 0 and hasattr(param, "ds_numel"):
           num_params = param.ds_numel


       all_param += num_params
       if param.requires_grad:
           trainable_params += num_params
   if use_4bit:
       trainable_params /= 2
   print(
       f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
   )

In [ ]:
dataset = load_dataset("HydraIndicLM/Hindi_Train_ClosedDomainQA")


cache_dir = "/media/anil/New Volume1/sumedha/OHmodel/model" # Model Location
os.makedirs(cache_dir,exist_ok=True)


model_name = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"
bnb_config = create_bnb_config() # Creating Configuration


model, tokenizer = load_model(model_name, bnb_config)


training_data = dataset["train"].shuffle().map(generate_and_tokenize_prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

Map:   0%|          | 0/20508 [00:00<?, ? examples/s]

In [ ]:
def train(model, tokenizer, dataset, output_dir):
  # Apply preprocessing to the model to prepare it by
  # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
  model.gradient_checkpointing_enable()


  # 2 - Using the prepare_model_for_kbit_training method from PEFT
  model = prepare_model_for_kbit_training(model)


  # Get lora module names
  modules = find_all_linear_names(model)
  print(modules)


  # Create PEFT config for these modules and wrap the model to PEFT
  peft_config = create_peft_config(modules)
  model = get_peft_model(model, peft_config)


  # Print information about the percentage of trainable parameters
  print_trainable_parameters(model)


  # Training parameters
  trainer = Trainer(
      model=model,
      train_dataset=training_data,
      args=TrainingArguments(
          num_train_epochs=2,
          per_device_train_batch_size=4,
          gradient_accumulation_steps=4,
          max_steps=10,
          learning_rate=2e-4,
          fp16=True,
          lr_scheduler_type ="cosine",
          logging_steps=10,
          warmup_ratio = 0.03,
          output_dir="outputs",
          optim='paged_adamw_32bit',
      ),
      data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
  )


  model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs


  ### SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py
  # Verifying the datatypes before training


  dtypes = {}
  for _, p in model.named_parameters():
      dtype = p.dtype
      if dtype not in dtypes: dtypes[dtype] = 0
      dtypes[dtype] += p.numel()
  total = 0
  for k, v in dtypes.items(): total+= v
  for k, v in dtypes.items():
      print(k, v, v/total)


  do_train = True


  # Launch training
  print("Training...")


  if do_train:
      train_result = trainer.train()
      metrics = train_result.metrics
      trainer.log_metrics("train", metrics)
      trainer.save_metrics("train", metrics)
      trainer.save_state()
      print(metrics)


  ###


  # Saving model
  print("Saving last checkpoint of the model...")
  os.makedirs(output_dir, exist_ok=True)
  trainer.model.save_pretrained(output_dir)


  # Free memory for merging weights
  # del model
  del trainer
  torch.cuda.empty_cache()
  import gc
  gc.collect()
  gc.collect()

In [ ]:
output_dir = "/content/drive/My Drive/llama2/final_checkpoint"
train(model, tokenizer, dataset, output_dir)

In [ ]:

prompt = "हाथी को अंग्रेजी में क्या कहते हैं?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


['हाथी को अंग्रेजी में क्या कहते हैं?\n संतुलित करें।\n \n \n \n \n \n \n \n \n']

In [ ]:
!pip install langchain transformers qdrant-client accelerate torch bitsandbytes



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
INDIC_NLP_RESOURCES = r"C:\Users\sumed\Desktop\Projects\llama_agent\indic_nlp_resources"

In [ ]:
from indicnlp import common

common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
from indicnlp import loader

loader.load()

In [ ]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

with open("test_text.txt", "r") as file:
    input_text= file.read()
# input_text = ""
remove_nuktas = False
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("hi")
output_text = normalizer.normalize(input_text)

print(input_text)
print()

print("Before normalization")

print(" ".join([hex(ord(c)) for c in input_text]))
print("Length: {}".format(len(input_text)))
print()
print("After normalization")
print(" ".join([hex(ord(c)) for c in output_text]))
print("Length: {}".format(len(output_text)))

तोड़ दिया था और उसके खंडहरों पर दक्षिण में कई राज्य उठ खड़े हुए थे। इससे
बहुत पहले, चौदहवीं शताब्दी के आरंभ में, दो बड़े राज्य कायम हुए थे-गुलबर्ग
जो बहमनी राज्य के नाम से प्रसिद्ध है और विजयनगर का हिंदू राज्य।

दिल्‍ली की तबाही के बाद उत्तरी भारत कमज़ोर पड़कर टुकड़ों में बँट
गया। दक्षिण भारत की स्थिति बेहतर थी और वहाँ के राज्यों में सबसे बड़ी
और शक्तिशाली रियासत विजयनगर थी। इस रियासत और नगर ने उत्तर के        
बहुत से हिंदू शरणार्थियों को आकर्षित किया। उपलब्ध वृत्तांतों से पता चलता
है कि शहर बहुत समृद्ध और सुंदर था।

जब दक्षिण में विजयनगर तरक्की कर रहा था, उस समय उत्तर की
पहाड़ियों से होकर एक और हमलावर दिल्ली के पास, पानीपत के प्रसिद्ध
मैदान में आया। उसने 526 ई. में दिल्‍ली के सिंहासन को जीत लिया। मध्य
एशिया के तैमूर वंश का यह तुर्क-मंगोल बाबर था। भारत में मुगल साम्राज्य
की नींव उसी ने डाली।

समन्वय और मिली-जुली संस्कृति का विकास
कबीर, गुरु नानक और अमीर खुसरो

भारत पर मुस्लिम आक्रमण की या भारत में मुस्लिम युग की बात करना
गलत और भ्रामक है। इस्लाम ने भारत पर आक्रमण नहीं किया, वह भारत
में कुछ सदियों के बाद आया। आक्रमण तुर्कों (महमूद) ने किया था,
अफ़गानों ने किया था और उसके बाद तुर्क-मंगोल या मुगल आक्रमण
हुआ। इनमें से बाद के दो आक्रमण महत्त्वपूर्ण थे। अफ़गानों को हम भारत
का सीमावर्ती समुदाय कह सकते हैं, जो भारत के लिए पूरी तरह अजनबी
भी नहीं माने जा सकते। उनके राजनीतिक शासन के काल को हिंद-
अफ़गान युग कहना चाहिए। मुगल भारत के लिए बाहर के और अजनबी
लोग थे, फिर भी वे भारतीय ढाँचे में बड़ी तेज़ी से समा गए और उन्होंने
हिंद-मुगल युग की शुरुआत की।

अफ़गान शासक और जो लोग उनके साथ आए थे वे भी भारत में समा
गए। उनके परिवारों का पूरी तरह भारतीयकरण हो गया। भारत को वे अपना

नयी समस्याएँ

In [ ]:
import re


def remove_long_spaces(text):
    # Define a regular expression pattern to match spaces longer than one newline
    pattern = re.compile(r" {2,}")

    # Replace the matched pattern with a single space
    return re.sub(pattern, " ", text)

mod_input_text=remove_long_spaces(input_text)
print(mod_input_text)

In [ ]:
from indicnlp.tokenize import sentence_tokenize


sentences = sentence_tokenize.sentence_split(mod_input_text, lang="hi")
for t in sentences:
    print(t)

In [ ]:
import fasttext as ft

# Loding model for Hindi.
embed_model = ft.load_model("wiki.hi.bin")

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
 chunk_size=1000,
 chunk_overlap=20,
 length_function=len,
 is_separator_regex=False,
)
text_path = "C:\\Users\\sumed\\Desktop\\Projects\\llama_agent"
loader = DirectoryLoader(text_path, glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
import fasttext as ft

# Loding model for Hindi.
embed_model = ft.load_model("wiki.hi.bin")

In [ ]:
df["embeddings"] = df["page_content"].apply(
    lambda x: (embed_model.get_sentence_vector(x)).tolist()
)

In [ ]:
df["id"] = range(1, len(df) + 1)

In [ ]:
payload = df[["page_content", "metadata"]].to_dict(orient="records")

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(location=":memory:")

In [ ]:
from qdrant_client.http import models

client.delete_collection(collection_name="hindi_collection")
client.create_collection(
    collection_name="hindi_collection",
    vectors_config=models.VectorParams(size=300, distance=models.Distance.COSINE),
)

ModuleNotFoundError: No module named 'qdrant_client'

In [ ]:
client.upsert(
    collection_name="hindi_collection",
    points=models.Batch(
        ids=df["id"].to_list(),
        payloads=payload,
        vectors=df["embeddings"].to_list(),
    ),
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
# preparing config for quantizing the model into 4 bits
# quantization_config = BitsAndBytesConfig(
#  load_in_4bit=True,
#  bnb_4bit_compute_dtype=torch.float16,
#  bnb_4bit_quant_type="nf4",
#  bnb_4bit_use_double_quant=True,
# )
# load the tokenizer and the quantized mistral model
# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# model_4bit = AutoModelForCausalLM.from_pretrained(
#  model_id,
#  device_map="auto",
#  quantization_config=quantization_config,)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# using HuggingFace's pipeline
pipeline = pipeline(
 "text-generation",
 model=model_4bit,
 tokenizer=tokenizer,
 use_cache=True,
 device_map="auto",
 max_new_tokens=5000,
 do_sample=True,
 top_k=1,
 temperature = 0.01,
 num_return_sequences=1,
 eos_token_id=tokenizer.eos_token_id,
 pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
def generate_text(question):
 # Searching for relevant hits in the 'speech_collection'
 hits = client.search(
 collection_name="hindi_collection",
 query_vector= embed_model.get_sentence_vector(question).tolist(),
 limit=10,
 )
 # Creating context from the hits
 context = ''
 for hit in hits:
    context += hit.payload['page_content'] + '\n'
 # Constructing the prompt
 prompt = f"""<s>[INST] आप एक सम्मानीय सहायक हैं। आपका काम नीचे दिए गए संदर्भ से प्रश्नों का उत्तर देना है।
 संदर्भ: {context}
 प्रश्न: {question} [/INST] </s>
 """
 # Generating text using the GPT model
 sequences = pipeline(
 prompt,
 do_sample=True,
 temperature=0.7,
 top_k=50,
 top_p=0.95,
 num_return_sequences=1,
 )
 return sequences[0]['generated_text']

In [ ]:
generate_text("मुझे गांधी के बारे में विस्तार से बताएं।")
